# Thuật toán Gomory giải quy hoạch nguyên

In [21]:
# /*==========================================================================================*\
# **                        _           _ _   _     _  _         _                            **
# **                       | |__  _   _/ | |_| |__ | || |  _ __ | |__                         **
# **                       | '_ \| | | | | __| '_ \| || |_| '_ \| '_ \                        **
# **                       | |_) | |_| | | |_| | | |__   _| | | | | | |                       **
# **                       |_.__/ \__,_|_|\__|_| |_|  |_| |_| |_|_| |_|                       **
# \*==========================================================================================*/

# DECLASSIFIED. Approved for release 2022/24/11 
# Author: Bùi Tiến Thành (@bu1th4nh)
# Date: 2022/11/21 21:50 
# CTTN Toán tin K64

from datetime import date
import pandas as pd
# import pandas_profiling as pp
import numpy as np
import tkinter
import scipy as sp
import sklearn as sk
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import json # make sense of what the requests return
from fractions import Fraction;
from IPython.display import display, Markdown


warnings.simplefilter('ignore')


## 1. Thuật toán

### 1.1. Xử lý bảng đơn hình

##### 1.1.1. Xoay bảng

In [22]:
def rotateTableau(m, n, drift, df_cur_table, idx_theta_min, idx_delta_max):
    A2 = df_cur_table.iloc[:m, drift-1:-1].to_numpy();
    A2[idx_theta_min,:] /= A2[idx_theta_min, idx_delta_max+1];
    for i in range(m):
        if(i == idx_theta_min): continue;
        A2[i,:] -= A2[i,idx_delta_max+1] * A2[idx_theta_min,:];
    x2 = np.array(A2[:,0]);
    A2 = A2[:,1:]
    return (A2, x2);

##### 1.1.2. Biến đổi bảng trước cắt - Thuật toán đơn hình cho bài toán nới lỏng

In [23]:
def oneStepRelaxedSimplex(A, b, c, starting_point):
    n = len(c); # Nr of variables
    m = len(b); # Nr of equations
    
    # Build tableau: Columns
    col_list = ["B_id", "B", "C_B", "X_B"];
    drift    = len(col_list);
    for i in range(n):
        col_list.append(("A" + str(i+1)));
    col_list.append("theta")
    df_table = pd.DataFrame(columns=col_list);


    # Build tableau: Rows
    for i in range(m):
        row = [int(starting_point[i]), "A{0}".format(starting_point[i]+1), c[starting_point[i]], b[i]];
        for Ariel in A[i]: row.append(Ariel);
        row.append(np.nan);
        df_table.loc[len(df_table.index)] = row;


    # Calculating delta
    padding = [np.nan, np.nan, np.nan, np.nan];         # Thêm padding cho dễ phân biệt hàng
    delta   = [np.nan, np.nan, np.nan, "Delta = ["];
    for i in range(n):
        padding.append(np.nan)
        delta.append(
            (np.array(df_table.iloc[:, i+drift]).T @ np.array(df_table["C_B"])) - c[i]
        );
    padding.append(np.nan);
    delta.append("]");
    df_table.loc[len(df_table.index)] = padding;         # Thêm padding cho dễ phân biệt hàng
    df_table.loc[len(df_table.index)] = delta;

    
    # Checking terminal condition
    if(np.nanmax(delta[drift:-1]) <= 0):
        # Found solution
        sol = [];
        for i in range(n): sol.append(Fraction(0)); 
        for i in range(m): 
            sol[int(df_table.loc[i, "B_id"])] = df_table.loc[i, "X_B"];
        return (True, df_table, sol)
    else:
        # Next table
        idx_delta_max = np.nanargmax(delta[drift:-1]);
        for i in range(m):
            z_js = df_table.iloc[i, drift+idx_delta_max]
            df_table.loc[i, "theta"] = df_table.loc[i, "X_B"] / z_js if z_js > 0 else np.inf;
        idx_theta_min = np.nanargmin(df_table["theta"][:-1]);

        (A2, x2) = rotateTableau(m, n, drift, df_table, idx_theta_min, idx_delta_max);
        starting_point[idx_theta_min] = idx_delta_max;
        
        
        # print(A2)
        # print("0-indexed")
        # print("Row: {0}".format(idx_theta_min));
        # print("Col: {0}".format(idx_delta_max));
        # print("1-indexed")
        # print("Row: {0}".format(idx_theta_min+1));
        # print("Col: {0}".format(idx_delta_max+1));
        

        return (False, df_table, x2, A2, starting_point)


##### 1.1.3. Biến đổi bảng sau cắt - Thuật toán đơn hình đối ngẫu

In [24]:
def oneStepDualSimplex(A, b, c, starting_point, integer_pos):
    n = len(c); # Nr of variables
    m = len(b); # Nr of equations
    
    # Build tableau: Columns
    col_list = ["B_id", "B", "C_B", "X_B"];
    drift    = len(col_list);
    for i in range(n):
        col_list.append(("A" + str(i+1)));
    col_list.append("theta")
    df_table = pd.DataFrame(columns=col_list);


    # Build tableau: Rows
    for i in range(m):
        row = [int(starting_point[i]), "A{0}".format(starting_point[i]+1), c[starting_point[i]], b[i]];
        for Ariel in A[i]: row.append(Ariel);
        row.append(np.nan);
        df_table.loc[len(df_table.index)] = row;


    # Calculating delta
    delta = [np.nan, np.nan, np.nan, "Delta = ["];
    padding = [np.nan, np.nan, np.nan, np.nan];         # Thêm padding cho dễ phân biệt hàng
    for i in range(n):
        padding.append(np.nan);
        delta.append(
            (np.array(df_table.iloc[:, i+drift]).T @ np.array(df_table["C_B"])) - c[i]
        );
    padding.append(np.nan);
    delta.append("]");
    df_table.loc[len(df_table.index)] = padding;         # Thêm padding cho dễ phân biệt hàng
    df_table.loc[len(df_table.index)] = delta;


    # Checking terminal condition
    check_term_cond = True;
    for i in range(m):
        if((starting_point[i] in integer_pos) and (df_table.loc[i, "X_B"].denominator != 1)):
            check_term_cond = False;
            break;
    if(check_term_cond):
        # Found solution
        sol = [];
        for i in range(n): sol.append(Fraction(0)); 
        for i in range(m): 
            sol[int(df_table.loc[i, "B_id"])] = df_table.loc[i, "X_B"];
        return (1, df_table, sol);
        

    # Calculating divs
    for i in range(m):
        if(df_table.loc[i, "X_B"] < 0):
            idx_chosen_row = i;
            idx_chosen_col = -1;
            divs = [np.nan, np.nan, np.nan, "Delta/Z = ["];

            for j in range(n):
                Ariel = df_table.iloc[i, drift+j];
                divs.append(delta[drift+j] / Ariel if Ariel < 0 else np.inf);
            divs.append("]");

            df_table.loc[len(df_table.index)] = divs;
            

            if(np.nanmin(divs[drift:-1]) < np.inf):
                idx_chosen_col = np.nanargmin(divs[drift:-1]);
            
            if(idx_chosen_col >= 0):
                (A2, x2) = rotateTableau(m, n, drift, df_table, idx_chosen_row, idx_chosen_col);
                starting_point[idx_chosen_row] = idx_chosen_col;
                return (0, df_table, x2, A2, starting_point)
                
    return (-1, df_table, b, A, starting_point)

### 1.2. Cắt Gomory

##### 1.2.1. Thêm và in cắt

In [25]:
def assignNewCut(A, b, c, starting_point, new_cut, f_star):
    n = len(c); # Nr of variables
    m = len(b); # Nr of equations

    Ariel = [];
    for _ in range(m): Ariel.append(Fraction(0));
    Ariel.append(Fraction(1));
    Ariel = np.reshape(np.array(Ariel), (m+1, 1));

    A = np.append(A, [new_cut], axis=0);
    A = np.append(A, Ariel, axis=1)
    b = np.append(b, -f_star);
    c = np.append(c, [Fraction(0)]);
    starting_point.append(n-1);
    n += 1;
    m += 1;

    return (A, b, c, n, m, starting_point)


def frac2Latex(fracc: Fraction):
    if(fracc.numerator == 1 and fracc.denominator == 1): return "";
    elif(fracc.denominator != 1): return "\\frac{" + "{0}".format(fracc.numerator)+ "}{" + "{0}".format(fracc.denominator) + "}";
    else: return "{0}".format(fracc.numerator);


def printCut(A, n, f_star):
    new_cut_str         = "`New cut - Text vers.:` ${0} = ".format(frac2Latex(-f_star));
    for i in range(n):
        new_cut_str += "{0}x_{1}".format(frac2Latex(A[-1][i]), i+1);
        if(i < n-1): new_cut_str += "  +  ";
    new_cut_str += "$"
    display(Markdown(new_cut_str));
    print()

    new_cut_nonzero     = "`New cut - Nonzero v.:` ${0} = ".format(frac2Latex(-f_star));
    for i in range(n):
        if(A[-1][i] == 0): continue;
        else: new_cut_nonzero += "{0}x_{1}  +  ".format(frac2Latex(A[-1][i]), i+1);
    new_cut_nonzero = new_cut_nonzero[:-4] + "$"
    display(Markdown(new_cut_nonzero));
    print()
    
    new_cut_ui_friendly = "New cut - Coeff only: {0} | ".format(-f_star);
    for i in range(n):
        new_cut_ui_friendly += "{0}".format(A[-1][i]);
        if(i < n-1): new_cut_ui_friendly += "   ";
    print(new_cut_ui_friendly);
    print()


##### 1.2.2. Quy hoạch nguyên hỗn hợp

In [26]:
# Mixed Integers
def MixedIntegerCut(A, b, c, starting_point, integer_pos):
    n = len(c); # Nr of variables
    m = len(b); # Nr of equations

    for i in range(m):
        if((starting_point[i] in integer_pos) and (b[starting_point[i]].denominator != 1)):
            print("Cut executed in row #{0}".format(i+1))

            new_cut = [];
            f_star  = b[i] - np.floor(b[i]);

            for j in range(n):
                if(starting_point[i] == j): new_cut.append(Fraction(0));
                elif(A[i, j] < 0): new_cut.append(-f_star / (f_star - Fraction(1)) * A[i, j]);
                else: new_cut.append(-A[i][j]);


            (A, b, c, n, m, starting_point) = assignNewCut(A, b, c, starting_point, new_cut, f_star)
            printCut(A, n, f_star)
            return (A, b, c, starting_point);
    raise Exception("Cut not found")

##### 1.2.3. Quy hoạch nguyên toàn phần

In [27]:
# Full Integers
def FullIntegerCut(A, b, c, starting_point):
    n = len(c); # Nr of variables
    m = len(b); # Nr of equations
    for i in range(m):
        if(b[i].denominator == 1): continue;
        print("Cut executed in row #{0}".format(i+1))
        
        new_cut = [];
        f_star      = b[i] - np.floor(b[i]);

        for j in range(n): new_cut.append(-(A[i][j] - np.floor(A[i][j])));


        (A, b, c, n, m, starting_point) = assignNewCut(A, b, c, starting_point, new_cut, f_star)
        printCut(A, n, f_star)
        return (A, b, c, starting_point);
    raise Exception("Cut not found")

### 1.3. Thuật toán lõi

##### 1.3.1. In thông số

In [28]:
def printSolution(sol):
    sol_str = "";
    for Ariel in sol:
        sol_str += "{0}".format(Ariel);
        sol_str += ";  "
    sol_str = sol_str[:-1];
    print("Solution: [{0}]".format(sol_str))

def printCoefficient(coeff):
    coeff_str = "";
    for Ariel in coeff:
        coeff_str += "{0}".format(Ariel);
        coeff_str += ";  "
    coeff_str = coeff_str[:-1];
    print("Coefficients: [{0}]".format(coeff_str))

##### 1.3.2. Thuật toán

In [29]:
def IntegerSimplex(A, b, c, starting_point, integer_type = 'full', integer_pos = None, penalty_index = None):
    tableau_id       = 0;
    simplex_id       = 0;
    dual_simplex_id  = 0;
    cut_id           = 0;
    
    # Simplex step for relaxed problem - Assume that the relaxed LP has solution
    while(True):
        # Debug
        tableau_id += 1;
        simplex_id += 1;
        print("Tableau #{0} - Simplex step #{1}:".format(tableau_id, simplex_id))

        # Simplex
        Ariel = oneStepRelaxedSimplex(A, b, c, starting_point);
        display(Ariel[1].fillna(' ').drop("B_id", axis='columns'))
        if(Ariel[0]): 
            print("Optimal Solution for Relaxed Problem found. Moving out to Gomory step")
            print("------------------------------------------------------------\n\n")
            break;
        else: (_, _, b, A, starting_point) = Ariel;


    
    # Checking penalty condition - Assume that x penalty coeffs have last x index in the tableau 
    if(penalty_index != None):
        if(len(set(starting_point).intersection(set(penalty_index))) > 0):
            raise Exception("No solution")
        A = A[:,:-len(penalty_index)];
        c = c[:-len(penalty_index)];



    # Cut step
    while(True):
        # Debug
        cut_id += 1;
        print("------------------------------------------------------------")
        print("Cut #{0}".format(cut_id))
        

        # Deliver a cut
        if(integer_type == 'full'):
            (A, b, c, starting_point) = FullIntegerCut(A, b, c, starting_point);
            integer_pos = range(0, len(c));
        elif(integer_type == 'mixed'):
            (A, b, c, starting_point) = MixedIntegerCut(A, b, c, starting_point, integer_pos)

        # Dual Simplex 
        print();
        dual_simplex_id = 0;
        while(True):
            # Debug
            tableau_id += 1;
            dual_simplex_id += 1;
            print("Tableau #{0} - Dual Simplex step #{1} of cut #{2}:".format(tableau_id, dual_simplex_id, cut_id))
            
            # Dual Simplex - One step
            Ariel = oneStepDualSimplex(A, b, c, starting_point, integer_pos);
            display(Ariel[1].fillna(' ').drop("B_id", axis='columns'))
            if(Ariel[0] == 1): 
                print("Optimal Solution found")
                printSolution(Ariel[2])
                return;
            elif(Ariel[0] == 0):
                (_, _, b, A, starting_point) = Ariel;
            else:
                (_, _, b, A, starting_point) = Ariel;
                break;

            # End marking
            # print("End of Dual Simplex step #{1} of cut #{2}:".format(tableau_id, dual_simplex_id, cut_id))

        # End marking of cut
        print("Cut #{0} ended.".format(cut_id));
        print("\n\n\n\n")


## 2. Thực hiện

### 2.0. Code Generation

In [30]:
n = 5; # Nr of variables (cols)
m = 2; # Nr of equations (rows)


c_str = "c = [\n";
for i in range(n): 
    c_str += "\tFraction(0)";
    if(i != n-1): c_str += ",";
    c_str += "\n";
c_str += "];\n"
print(c_str)


A_str = "A = [\n";
for i in range(m):
    A_str += "\t[";
    for j in range(n):
        A_str += "Fraction(0)";
        if(j != n-1): A_str += ",   ";
    A_str += "]"
    if(i != m-1): A_str += ",";
    A_str += "\n"
A_str += "];\n"
print(A_str)


b_str = "b = [\n";
for i in range(m): 
    b_str += "\tFraction(0)";
    if(i != m-1): b_str += ",";
    b_str += "\n";
b_str += "];\n"
print(b_str)




c = [
	Fraction(0),
	Fraction(0),
	Fraction(0),
	Fraction(0),
	Fraction(0)
];

A = [
	[Fraction(0),   Fraction(0),   Fraction(0),   Fraction(0),   Fraction(0)],
	[Fraction(0),   Fraction(0),   Fraction(0),   Fraction(0),   Fraction(0)]
];

b = [
	Fraction(0),
	Fraction(0)
];



### 2.1. Ví dụ giải

#### Ví dụ test quy hoạch nguyên toàn phần - 2 cắt

In [31]:
# Ví dụ test quy hoạch nguyên toàn phần - 2 cắt

c = [
	Fraction(-3),
	Fraction(-11),
	Fraction(0),
	Fraction(0)
];

A = [
	[Fraction(2),   Fraction(6),   Fraction(1),   Fraction(0)],
	[Fraction(6),   Fraction(4),   Fraction(0),   Fraction(1)]
];

b = [
	Fraction(7),
	Fraction(15)
];

starting_point = [
    2,
    3
];

# IntegerSimplex(A, b, c, starting_point, integer_type = "mixed", integer_pos = needed_integer_pos);

IntegerSimplex(A, b, c, starting_point, integer_type = "full", penalty_index = None);

Tableau #1 - Simplex step #1:


,B,C_B,X_B,A1,A2,A3,A4,theta
0,A3,0,7,2,6,1,0,7/6
1,A4,0,15,6,4,0,1,15/4
2,,,,,,,,
3,,,Delta = [,3,11,0,0,]


Tableau #2 - Simplex step #2:


,B,C_B,X_B,A1,A2,A3,A4,theta
0,A2,-11,7/6,1/3,1,1/6,0,
1,A4,0,31/3,14/3,0,-2/3,1,
2,,,,,,,,
3,,,Delta = [,-2/3,0,-11/6,0,]


Optimal Solution for Relaxed Problem found. Moving out to Gomory step
------------------------------------------------------------


------------------------------------------------------------
Cut #1
Cut executed in row #1


...


New cut - Coeff only: -1/2 | 0   0   -1/2   0   0   1


Tableau #5 - Dual Simplex step #1 of cut #2:


,B,C_B,X_B,A1,A2,A3,A4,A5,A6,theta
0,A2,-11,1,0,1,0,0,1,0,
1,A4,0,8,0,0,-3,1,14,0,
2,A1,-3,1/2,1,0,1/2,0,-3,0,
3,A5,0,-1/2,0,0,-1/2,0,0,1,
4,,,,,,,,,,
5,,,Delta = [,0,0,-3/2,0,-2,0,]
6,,,Delta/Z = [,inf,inf,3,inf,inf,inf,]


Tableau #6 - Dual Simplex step #2 of cut #2:


,B,C_B,X_B,A1,A2,A3,A4,A5,A6,theta
0,A2,-11,1,0,1,0,0,1,0,
1,A4,0,11,0,0,0,1,14,-6,
2,A1,-3,0,1,0,0,0,-3,1,
3,A3,0,1,0,0,1,0,0,-2,
4,,,,,,,,,,
5,,,Delta = [,0,0,0,0,-2,-3,]


Optimal Solution found
Solution: [0;  1;  1;  11;  0;  0; ]


#### Ví dụ test quy hoạch nguyên toàn phần + bài toán phạt

In [32]:
# Ví dụ test quy hoạch nguyên toàn phần + bài toán phạt

c = [
	Fraction(-3, 28),
	Fraction(1),
	Fraction(0),
	Fraction(0),
	Fraction(1000000)
];

A = [
	[Fraction(1),   Fraction(-1),   Fraction(1),   Fraction(0),   Fraction(0)],
	[Fraction(1),   Fraction(2),   Fraction(0),   Fraction(-1),   Fraction(1)]
];

b = [
	Fraction(1),
	Fraction(5)
];

starting_point = [
    2,
    4
];

penalty = [
	4
]

# IntegerSimplex(A, b, c, starting_point, integer_type = "mixed", integer_pos = needed_integer_pos);

IntegerSimplex(A, b, c, starting_point, integer_type = "full", penalty_index = penalty);

Tableau #1 - Simplex step #1:


,B,C_B,X_B,A1,A2,A3,A4,A5,theta
0,A3,0,1,1,-1,1,0,0,inf
1,A5,1000000,5,1,2,0,-1,1,5/2
2,,,,,,,,,
3,,,Delta = [,28000003/28,1999999,0,-1000000,0,]


Tableau #2 - Simplex step #2:


,B,C_B,X_B,A1,A2,A3,A4,A5,theta
0,A3,0,7/2,3/2,0,1,-1/2,1/2,7/3
1,A2,1,5/2,1/2,1,0,-1/2,1/2,5
2,,,,,,,,,
3,,,Delta = [,17/28,0,0,-1/2,-1999999/2,]


Tableau #3 - Simplex step #3:


...


New cut - Coeff only: -1/2 | 0   0   0   0   -1/2   1


Tableau #6 - Dual Simplex step #1 of cut #2:


,B,C_B,X_B,A1,A2,A3,A4,A5,A6,theta
0,A1,-3/28,5/2,1,0,1,0,-1/2,0,
1,A2,1,3/2,0,1,0,0,-1/2,0,
2,A4,0,1/2,0,0,1,1,-3/2,0,
3,A5,0,-1/2,0,0,0,0,-1/2,1,
4,,,,,,,,,,
5,,,Delta = [,0,0,-3/28,0,-25/56,0,]
6,,,Delta/Z = [,inf,inf,inf,inf,25/28,inf,]


Tableau #7 - Dual Simplex step #2 of cut #2:


,B,C_B,X_B,A1,A2,A3,A4,A5,A6,theta
0,A1,-3/28,3,1,0,1,0,0,-1,
1,A2,1,2,0,1,0,0,0,-1,
2,A4,0,2,0,0,1,1,0,-3,
3,A5,0,1,0,0,0,0,1,-2,
4,,,,,,,,,,
5,,,Delta = [,0,0,-3/28,0,0,-25/28,]


Optimal Solution found
Solution: [3;  2;  0;  2;  1;  0; ]


#### Ví dụ test quy hoạch nguyên toàn phần + bài toán phạt

In [33]:
# Ví dụ test quy hoạch nguyên toàn phần + bài toán phạt

c = [
	Fraction(-3, 28),
	Fraction(1),
	Fraction(0),
	Fraction(0),
	Fraction(1000000)
];

A = [
	[Fraction(1),   Fraction(-1),   Fraction(1),   Fraction(0),   Fraction(0)],
	[Fraction(1),   Fraction(2),   Fraction(0),   Fraction(-1),   Fraction(1)]
];

b = [
	Fraction(1),
	Fraction(6)
];

starting_point = [
    2,
    4
];

penalty = [
	4
]

# IntegerSimplex(A, b, c, starting_point, integer_type = "mixed", integer_pos = needed_integer_pos);

IntegerSimplex(A, b, c, starting_point, integer_type = "full", penalty_index = penalty);

Tableau #1 - Simplex step #1:


,B,C_B,X_B,A1,A2,A3,A4,A5,theta
0,A3,0,1,1,-1,1,0,0,inf
1,A5,1000000,6,1,2,0,-1,1,3
2,,,,,,,,,
3,,,Delta = [,28000003/28,1999999,0,-1000000,0,]


Tableau #2 - Simplex step #2:


,B,C_B,X_B,A1,A2,A3,A4,A5,theta
0,A3,0,4,3/2,0,1,-1/2,1/2,8/3
1,A2,1,3,1/2,1,0,-1/2,1/2,6
2,,,,,,,,,
3,,,Delta = [,17/28,0,0,-1/2,-1999999/2,]


Tableau #3 - Simplex step #3:


...


New cut - Coeff only: -2/3 | 0   0   -2/3   -2/3   1


Tableau #4 - Dual Simplex step #1 of cut #1:


,B,C_B,X_B,A1,A2,A3,A4,A5,theta
0,A1,-3/28,8/3,1,0,2/3,-1/3,0,
1,A2,1,5/3,0,1,-1/3,-1/3,0,
2,A4,0,-2/3,0,0,-2/3,-2/3,1,
3,,,,,,,,,
4,,,Delta = [,0,0,-17/42,-25/84,0,]
5,,,Delta/Z = [,inf,inf,17/28,25/56,inf,]


Tableau #5 - Dual Simplex step #2 of cut #1:


,B,C_B,X_B,A1,A2,A3,A4,A5,theta
0,A1,-3/28,3,1,0,1,0,-1/2,
1,A2,1,2,0,1,0,0,-1/2,
2,A4,0,1,0,0,1,1,-3/2,
3,,,,,,,,,
4,,,Delta = [,0,0,-3/28,0,-25/56,]


Optimal Solution found
Solution: [3;  2;  0;  1;  0; ]


#### Ví dụ test quy hoạch nguyên toàn phần 20 bảng

In [34]:
# Ví dụ test quy hoạch nguyên toàn phần 20 bảng

c = [
	Fraction(-3),
	Fraction(-4),
	Fraction(0),
	Fraction(0)
];

A = [
	[Fraction(3),   Fraction(2),   Fraction(1),   Fraction(0)],
	[Fraction(1),   Fraction(4),   Fraction(0),   Fraction(1)]
];

b = [
	Fraction(8),
	Fraction(10)
];

starting_point = [
    2,
    3
];


#### Ví dụ test mix integer thông thường - bộ số $\max 6 * x_1 + 8 * x_2 | \min - 6 * x_1 - 8 * x_2$, $x_1$ nguyên

In [35]:
# Ví dụ test mix integer thông thường - bộ số max 6 * x_1 + 8 * x_2 | min - 6 * x_1 - 8 * x_2, x1 nguyên

c = [
	Fraction(-6),
	Fraction(-8),
	Fraction(0),
	Fraction(0)
];

A = [
	[Fraction(-1, 3),   Fraction(1),   Fraction(1),   Fraction(0)],
	[Fraction(1),   Fraction(1, 7),   Fraction(0),   Fraction(1)]
];

b = [
	Fraction(2),
	Fraction(5)
];

starting_point = [
    2,
    3
];

needed_integer_pos = [
	0
];

IntegerSimplex(A, b, c, starting_point, integer_type = "mixed", integer_pos = needed_integer_pos);

# IntegerSimplex(A, b, c, starting_point, integer_type = "full", penalty_index = None);


Tableau #1 - Simplex step #1:


,B,C_B,X_B,A1,A2,A3,A4,theta
0,A3,0,2,-1/3,1,1,0,2
1,A4,0,5,1,1/7,0,1,35
2,,,,,,,,
3,,,Delta = [,6,8,0,0,]


Tableau #2 - Simplex step #2:


,B,C_B,X_B,A1,A2,A3,A4,theta
0,A2,-8,2,-1/3,1,1,0,inf
1,A4,0,33/7,22/21,0,-1/7,1,9/2
2,,,,,,,,
3,,,Delta = [,26/3,0,-8,0,]


Tableau #3 - Simplex step #3:


...


New cut - Coeff only: -1/2 | 0   0   -3/22   -21/22   1


Tableau #4 - Dual Simplex step #1 of cut #1:


,B,C_B,X_B,A1,A2,A3,A4,A5,theta
0,A2,-8,7/2,0,1,21/22,7/22,0,
1,A1,-6,9/2,1,0,-3/22,21/22,0,
2,A4,0,-1/2,0,0,-3/22,-21/22,1,
3,,,,,,,,,
4,,,Delta = [,0,0,-75/11,-91/11,0,]
5,,,Delta/Z = [,inf,inf,50,26/3,inf,]


Tableau #5 - Dual Simplex step #2 of cut #1:


,B,C_B,X_B,A1,A2,A3,A4,A5,theta
0,A2,-8,10/3,0,1,10/11,0,1/3,
1,A1,-6,4,1,0,-3/11,0,1,
2,A4,0,11/21,0,0,1/7,1,-22/21,
3,,,,,,,,,
4,,,Delta = [,0,0,-62/11,0,-26/3,]


Optimal Solution found
Solution: [4;  10/3;  0;  11/21;  0; ]


#### Ví dụ test mix integer thông thường - bộ số  $\max 5 * x_1 + 6 * x_2 | \min - 5 * x_1 - 6 * x_2$, $x_1$ nguyên

In [36]:
# Ví dụ test mix integer thông thường - bộ số max 5 * x_1 + 6 * x_2 | min - 5 * x_1 - 6 * x_2, x1 nguyên

c = [
	Fraction(-5),
	Fraction(-6),
	Fraction(0),
	Fraction(0)
];

A = [
	[Fraction(10),   Fraction(3),   Fraction(1),   Fraction(0)],
	[Fraction(2),   Fraction(3),   Fraction(0),   Fraction(1)]
];

b = [
	Fraction(52),
	Fraction(18)
];

starting_point = [
    2,
    3
];

needed_integer_pos = [
	0
];


IntegerSimplex(A, b, c, starting_point, integer_type = "mixed", integer_pos = needed_integer_pos);

Tableau #1 - Simplex step #1:


,B,C_B,X_B,A1,A2,A3,A4,theta
0,A3,0,52,10,3,1,0,52/3
1,A4,0,18,2,3,0,1,6
2,,,,,,,,
3,,,Delta = [,5,6,0,0,]


Tableau #2 - Simplex step #2:


,B,C_B,X_B,A1,A2,A3,A4,theta
0,A3,0,34,8,0,1,-1,17/4
1,A2,-6,6,2/3,1,0,1/3,9
2,,,,,,,,
3,,,Delta = [,1,0,0,-2,]


Tableau #3 - Simplex step #3:


...


New cut - Coeff only: -1/4 | 0   0   -1/8   -1/24   1


Tableau #4 - Dual Simplex step #1 of cut #1:


,B,C_B,X_B,A1,A2,A3,A4,A5,theta
0,A1,-5,17/4,1,0,1/8,-1/8,0,
1,A2,-6,19/6,0,1,-1/12,5/12,0,
2,A4,0,-1/4,0,0,-1/8,-1/24,1,
3,,,,,,,,,
4,,,Delta = [,0,0,-1/8,-15/8,0,]
5,,,Delta/Z = [,inf,inf,1,45,inf,]


Tableau #5 - Dual Simplex step #2 of cut #1:


,B,C_B,X_B,A1,A2,A3,A4,A5,theta
0,A1,-5,4,1,0,0,-1/6,1,
1,A2,-6,10/3,0,1,0,4/9,-2/3,
2,A3,0,2,0,0,1,1/3,-8,
3,,,,,,,,,
4,,,Delta = [,0,0,0,-11/6,-1,]


Optimal Solution found
Solution: [4;  10/3;  2;  0;  0; ]


#### Ví dụ test mix integer có kèm phân số - bộ số  $\max x_1 + 17/56 * x_2 | \min -x_1 - 17/56 * x_2$, $x_1$ nguyên

In [37]:
# Ví dụ test mix integer có kèm phân số - bộ số max x_1 + 17/56 * x_2 | min -x_1 - 17/56 * x_2, x1 nguyên

c = [
	Fraction(-1),
	Fraction(-17, 56),
	Fraction(0),
	Fraction(0)
];

A = [
	[Fraction(1),   Fraction(1),   Fraction(1),   Fraction(0)],
	[Fraction(3),   Fraction(-2),   Fraction(0),   Fraction(1)]
];

b = [
	Fraction(7, 2),
	Fraction(13, 2)
];

starting_point = [
    2,
    3
];

needed_integer_pos = [
	0
];

IntegerSimplex(A, b, c, starting_point, integer_type = "mixed", integer_pos = needed_integer_pos);

# IntegerSimplex(A, b, c, starting_point, integer_type = "full", penalty_index = None);

Tableau #1 - Simplex step #1:


,B,C_B,X_B,A1,A2,A3,A4,theta
0,A3,0,7/2,1,1,1,0,7/2
1,A4,0,13/2,3,-2,0,1,13/6
2,,,,,,,,
3,,,Delta = [,1,17/56,0,0,]


Tableau #2 - Simplex step #2:


,B,C_B,X_B,A1,A2,A3,A4,theta
0,A3,0,4/3,0,5/3,1,-1/3,4/5
1,A1,-1,13/6,1,-2/3,0,1/3,inf
2,,,,,,,,
3,,,Delta = [,0,163/168,0,-1/3,]


Tableau #3 - Simplex step #3:


...


New cut - Coeff only: -7/10 | 0   0   -2/5   -1/5   1


Tableau #4 - Dual Simplex step #1 of cut #1:


,B,C_B,X_B,A1,A2,A3,A4,A5,theta
0,A2,-17/56,4/5,0,1,3/5,-1/5,0,
1,A1,-1,27/10,1,0,2/5,1/5,0,
2,A4,0,-7/10,0,0,-2/5,-1/5,1,
3,,,,,,,,,
4,,,Delta = [,0,0,-163/280,-39/280,0,]
5,,,Delta/Z = [,inf,inf,163/112,39/56,inf,]


Tableau #5 - Dual Simplex step #2 of cut #1:


,B,C_B,X_B,A1,A2,A3,A4,A5,theta
0,A2,-17/56,3/2,0,1,1,0,-1,
1,A1,-1,2,1,0,0,0,1,
2,A4,0,7/2,0,0,2,1,-5,
3,,,,,,,,,
4,,,Delta = [,0,0,-17/56,0,-39/56,]


Optimal Solution found
Solution: [2;  3/2;  0;  7/2;  0; ]


#### Ví dụ test mix integer 6 biến + bài toán phạt với $x_2$ nguyên

In [38]:
# Ví dụ test mix integer 6 biến + bài toán phạt với x2 nguyên

c = [
	Fraction(-4),
	Fraction(-3),
	Fraction(0),
	Fraction(0)
];

A = [
	[Fraction(2),   Fraction(1),   Fraction(1),   Fraction(0)],
	[Fraction(-1),   Fraction(2),   Fraction(0),   Fraction(1)]
];

b = [
	Fraction(11),
	Fraction(6)
];

starting_point = [
    2,
    3 
];



IntegerSimplex(A, b, c, starting_point, integer_type = "full")

Tableau #1 - Simplex step #1:


,B,C_B,X_B,A1,A2,A3,A4,theta
0,A3,0,11,2,1,1,0,11/2
1,A4,0,6,-1,2,0,1,inf
2,,,,,,,,
3,,,Delta = [,4,3,0,0,]


Tableau #2 - Simplex step #2:


,B,C_B,X_B,A1,A2,A3,A4,theta
0,A1,-4,11/2,1,1/2,1/2,0,11
1,A4,0,23/2,0,5/2,1/2,1,23/5
2,,,,,,,,
3,,,Delta = [,0,1,-2,0,]


Tableau #3 - Simplex step #3:


...


New cut - Coeff only: -1/2 | 0   0   0   0   0   0   -1/2   1


Tableau #10 - Dual Simplex step #1 of cut #4:


,B,C_B,X_B,A1,A2,A3,A4,A5,A6,A7,A8,theta
0,A1,-4,7/2,1,0,1,0,0,0,-1/2,0,
1,A2,-3,4,0,1,-1,0,0,0,1,0,
2,A4,0,3/2,0,0,3,1,0,0,-5/2,0,
3,A5,0,1,0,0,2,0,1,0,-2,0,
4,A6,0,1/2,0,0,1,0,0,1,-3/2,0,
5,A7,0,-1/2,0,0,0,0,0,0,-1/2,1,
6,,,,,,,,,,,,
7,,,Delta = [,0,0,-1,0,0,0,-1,0,]
8,,,Delta/Z = [,inf,inf,inf,inf,inf,inf,2,inf,]


Tableau #11 - Dual Simplex step #2 of cut #4:


,B,C_B,X_B,A1,A2,A3,A4,A5,A6,A7,A8,theta
0,A1,-4,4,1,0,1,0,0,0,0,-1,
1,A2,-3,3,0,1,-1,0,0,0,0,2,
2,A4,0,4,0,0,3,1,0,0,0,-5,
3,A5,0,3,0,0,2,0,1,0,0,-4,
4,A6,0,2,0,0,1,0,0,1,0,-3,
5,A7,0,1,0,0,0,0,0,0,1,-2,
6,,,,,,,,,,,,
7,,,Delta = [,0,0,-1,0,0,0,0,-2,]


Optimal Solution found
Solution: [4;  3;  0;  4;  3;  2;  1;  0; ]


# Credit

In [39]:
import webbrowser as brws
# brws.open("https://www.twitch.tv/esl_csgo")
# brws.open("https://princess.disney.com/cinderella")
# brws.open("https://princess.disney.com/")


# Code by bu1th4nh and Cinderella
# Powered, inspired and motivated by EDM, Counter-Strike:Global Offensive and Disney Princesses